# EEG Analysis

In this notebook we analyze the difference between incongruent and congruent trials in the MSIT EEG data. We look at both the difference in the event-related potentials and in time frequency power.

# Event-Related Potentials

Here we analyze the difference between incongruent and congruent event-related potentials (ERPs). ERPs are simply the time domain average of every stimulus presentation and response made. Averaging separately by each condition produces two waveforms that we can compare.

## Create the Evoked Data

In this cell we create the evoked (ERP) data by averaging the epochs data. We make four sets of evoked data. For both the average referenced data and the laplacian transformed data we create evoked data time-locked to the stimulus presentations and the responses.

In [1]:
import os
import sys
import json
sys.path.append('../src')
from utils import select_subjects
import pandas as pd
from mne import read_epochs, grand_average, write_evokeds, set_log_level
import numpy as np

set_log_level('critical')

# input and output directories
preproc_dir = '../data/derivatives/eeg_preprocessing'
deriv_dir = '../data/derivatives/eeg_analysis'
if not os.path.exists(deriv_dir):
    os.makedirs(deriv_dir)

# select out the subjects
subjects = select_subjects('both')

# load config parameters
config = json.load(open('experiment_config.json', 'r'))
epochs_info = config['epochs_info']

for trans_type in ['base', 'laplacian']:
    print(trans_type)

    for epo_time, epo_type in zip(epochs_info['epo_boundaries'],
                                  epochs_info['epo_types']):
        print(epo_type)
        group_evo = {'incongruent': [], 'congruent': []}
        group_data = {'incongruent': [], 'congruent': []}
            
        for subject in subjects:

            # load epochs
            if trans_type == 'laplacian':
                f = '%s/%s/%s_task-msit_%s_laplacian-epo.fif' 
            else:
                f = '%s/%s/%s_task-msit_%s-epo.fif'
            epo = read_epochs(f % (preproc_dir, subject, subject,
                                   epo_type), verbose=False)

            # crop filter buffer
            epo.crop(epo_time[0], epo_time[1])
            
            # downsample
            epo.decimate(5)
            
            # interpolate bads
            epo.interpolate_bads(reset_bads=True)
            
            # calculate evokeds
            evos = [epo[c].average() for c in config['event_types']]

            # write evokeds
            if not os.path.exists('%s/%s' % (deriv_dir, subject)):
                os.makedirs('%s/%s' % (deriv_dir, subject))
            if trans_type == 'laplacian':
                f = '%s/%s/%s_task-msit_%s_laplacian-ave.fif' 
            else:
                f = '%s/%s/%s_task-msit_%s-ave.fif'
            write_evokeds(f % (deriv_dir, subject, subject,
                               epo_type), evos) 
            
            # accumulate for group average
            for i, c in enumerate(config['event_types']):
                evos[i].interpolate_bads(reset_bads=True)
                group_evo[c].append(evos[i])
                group_data[c].append(evos[i].data * 1e6)
                
        # calculate and save group evokeds object
        evos = [grand_average(group_evo[c]) for c in config['event_types']]
        if not os.path.exists('%s/group' %  deriv_dir):
            os.makedirs('%s/group' %  deriv_dir)
        if trans_type == 'laplacian':
            f = '%s/group/group_task-msit_%s_laplacian-ave.fif' 
        else:
            f = '%s/group/group_task-msit_%s-ave.fif'
        write_evokeds(f % (deriv_dir, epo_type), evos) 
        
        # calculate and save group evokeds array
        data = np.array([group_data[c] for c in config['event_types']])
        if trans_type == 'laplacian':
            f = '%s/group/group_task-msit_%s_laplacian-ave.npz' 
        else:
            f = '%s/group/group_task-msit_%s-ave.npz'
        np.savez_compressed(f % (deriv_dir, epo_type), data=data,
                            conditions=config['event_types'],
                            times=evos[0].times, chs=evos[0].ch_names,
                            subjects=subjects)
            
print('Done!')

base
stimulus
response
laplacian
stimulus
response
Done!


## ERP TFCE Permutation Testing 

To look at differences between the ERPs for each condition we use <a href="https://www.ncbi.nlm.nih.gov/pubmed/23123297">Threshold Free Cluster Enhancement (TFCE)</a> <a href="https://www.ncbi.nlm.nih.gov/pubmed/17517438">spatiotemporal permutation testing</a>. This approach is very similar to the approach taken with the fMRI except that we now cluster over both time and space as opposed to just space in the fMRI. The technique is performed as follows: 

1. Our data consists of a # subjects x # channels x # time points array for each condition. We perform a paired t-test for every channel and time point pairing to get a # channels x # time points t-statistic array.
2. We use TFCE to pre-enhance the cluster structure of the data by applying the TFCE transform to the t-statistic array.
3. Next we permute the data by randomly flipping the signs (and hence the condition labels) of a random subset of subject's condition data. We perform a paired t-test on this permuted data to get a new t-statistic array.
4. We use TFCE to enhance the cluster structure of the permuted t-statistics and then we extract the TFCE value with the greatest absolute value to control for multiple comparisons.
5. We repeat steps 3 and 4 5000 times to generate a permutation null distribution consisting of the maximum TFCE value across all permutations.
6. The permutation p-values are calculated as the percentage of permutation values whose absolute value are greater than the non-permuted TFCE absolute value. This is done for every channel, time point pairing resulting in a # channels x # time points p value array.

We complete this process separately for both the stimulus and response epochs with and without the laplacian transform. We use <a href="https://www.martinos.org/mne/stable/generated/mne.stats.spatio_temporal_cluster_1samp_test.html#mne.stats.spatio_temporal_cluster_1samp_test"> MNE's implementation of TFCE spatiotemporal clustering</a>.

In [5]:
import os
import sys
import json
sys.path.append('../src')
from utils import select_subjects
import pandas as pd
import numpy as np
from mne import read_evokeds
from mne.channels import find_ch_connectivity
from mne.stats import spatio_temporal_cluster_1samp_test as stcluster_test

np.random.seed(10)

# input and output directories
deriv_dir = '../data/derivatives/eeg_analysis'

# select out the subjects
subjects = select_subjects('both')

# get the channel connectivity
f = '%s/group/group_task-msit_stimulus_laplacian-ave.fif'
evo = read_evokeds(f % deriv_dir, verbose=False)[0]
connectivity, ch_names = find_ch_connectivity(evo.info, 
                                              ch_type='eeg')

# load config parameters
config = json.load(open('experiment_config.json', 'r'))
epochs_info = config['epochs_info']
threshold = config['tfce_threshold'] 
num_perm = config['num_eeg_perm']

for trans_type in ['base', 'laplacian']:
    print(trans_type)
    
    for epo_type in epochs_info['epo_types']:
        print(epo_type)


        # extract the data (incongruent - congruent)
        if trans_type == 'laplacian':
            f = '%s/group/group_task-msit_%s_laplacian-ave.npz'
        else:
            f = '%s/group/group_task-msit_%s-ave.npz'
            
        data = np.load(f % (deriv_dir, epo_type))['data']
        data = data[0] - data[1]
        data = np.swapaxes(data, 1, 2)

        # run threshold free cluster enhancement permutation testing
        tfce, _, p_vals, perm_dist = stcluster_test(data, verbose=True,
                                                    threshold=threshold,
                                                    n_permutations=num_perm,
                                                    connectivity=connectivity,
                                                    seed=5, n_jobs=10) 
        p_vals = np.array(p_vals).reshape(tfce.shape)

        # save
        if trans_type == 'laplacian':
            f = '%s/group/group_task-msit_%s_laplacian_tfce.npz'
        else:
            f = '%s/group/group_task-msit_%s_tfce.npz'
        f = f % (deriv_dir, epo_type)
        np.savez_compressed(f, tfce=tfce.T, perm_dist=perm_dist, 
                            chs=evo.ch_names, times=evo.times, 
                            p_vals=p_vals.T)

print('Done!')

base
stimulus
stat_fun(H1): min=-7.270792 max=8.365770
Running initial clustering
Using 84 thresholds from 0.00 to 8.30 for TFCE computation (h_power=2.00, e_power=0.50)
Found 31570 clusters
Permuting 4999 times...
[......................................  ] 96.19238 \    

[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed: 101.0min remaining: 235.8min


[......................................  ] 96.19238 \    

[Parallel(n_jobs=10)]: Done   6 out of  10 | elapsed: 105.4min remaining: 70.3min


Computing cluster p-values


...

[......................................  ] 96.19238 \    

[Parallel(n_jobs=10)]: Done   6 out of  10 | elapsed: 111.0min remaining: 74.0min


Computing cluster p-values


[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed: 111.6min finished


Done.
Done!


## Visualize ERPS 

Here we visualize the group ERPs along with the statistical results. The cell below lets you interactively explore the data. Each parameter setting produces the following:
1. An ERP waveform plot on the left:
    - The incongruent and congruent conditions are plotted separately with standard error of the mean shading. 
    - The RT distributions are plotted in the background for the stimulus-locked data. 
    - Signficant time points for that channel are denoted with a shaded grey background
    - A dotted line indicates the time point plotted for the topomap to the right
2. An ERP topomap on the right:
    - This is the mean incongruent - congruent topomap signal plotted for just the chosen time point
    - Colorbar limits are hard to see. They vary from +- 7 uV/mm^2 for laplacian transform to +- 1 uV for non-laplacian transform
    - Significant sensors for that timepoint have their names plotted in bold on the topomap

Use the following parameter controls to explore the data:
- Use the ch parameter to change which channel's ERPs are plotted on the left
- Use the trans_type and epo_type parameters to change what type of ERPs are plotted
- Use time_ix parameter to change what time point is plotted for the topomap
- Use the threshold parameter to change the significance highlighting

In [4]:
import sys
sys.path.append('../src')
from utils import select_subjects
from eeg import visualize_erps, CH_NAMES
from ipywidgets import interact, fixed
import seaborn as sns
import pandas as pd
sns.set(font_scale=2, style='whitegrid')

%matplotlib inline

deriv_dir = '../data/derivatives/eeg_analysis'

# load behavior
behavior = pd.read_csv('../data/derivatives/behavior/group_data.tsv',
                       sep='\t', na_values='n/a')

interact(visualize_erps, deriv_dir=fixed(deriv_dir),
         trans_type=['laplacian', 'base'], epo_type=['stimulus', 'response'], 
         time_ix=(0, 450), ch=CH_NAMES,
         threshold=[.001, .01, .05], behavior=fixed(behavior));

aW50ZXJhY3RpdmUoY2hpbGRyZW49KERyb3Bkb3duKGRlc2NyaXB0aW9uPXUndHJhbnNfdHlwZScsIG9wdGlvbnM9KCdsYXBsYWNpYW4nLCAnYmFzZScpLCB2YWx1ZT0nbGFwbGFjaWFuJyksIETigKY=


## Summary of ERP Results

We see quite reasonable alignment with <a href="https://www.ncbi.nlm.nih.gov/pubmed/28220517">Gonazalez-Villar and Carillo-de-la-Pena, 2017</a>, one of the few published investigations of EEG in the MSIT task that we use as a reference:
- Lower N2 in Cz for incongruent compared to incongruent trials, though the effect is smaller. The midfrontal N2 is often thought of as a marker of cognitive control.
- Longer latency and smeared P3 over Pz for incongruent compared to incongruent trials. The P3 is often conceptualized as a marker of mental workload and thus this difference makes sense as the incongruent trials response time distribution extends out past the congruent distribution.

Potential Alignment with Wald RT Model:
- The Cz N2 effect occurs right around 300-350 ms. This is about equal to the average non-decision times across subjects (~300 ms). One interpretation is that maybe this N2 difference reflects a trial type categorization process and the lower N2 could be a signature of the increased decision boundary we observed in the model for incongruent trials being sent. The rest of the waveforms from this point on also look like two separate diffusion processes where the incongruent one is slower to reach the same threshold due to the lower starting point.
- In the response locked Pz data, the difference in ramping seems similar to a potential difference in drift rate with the incongruent having a lower drift rate as found in the behavior model.

### Potential Concerns

As with the fMRI, the full scalp TFCE method had a similar issue where a lot of the positive activation got lumped into one giant cluster. As a result, the statistics are probably quite misleading and especially a lot of the negative differences got wrongly ignored. I would refrain from interpreting the statistical shading.

### Future Directions

- Fix the statistics. Figure out how to reduce sensitivity of cluster methods to excessively large clusters or do more pointed hypothesis led statistics.
- Look at behavior model correlations with ERPs. Perhaps Cz N2 amplitude difference is correlated with the difference in decision boundary across subjects. Or perhaps the P3 response aligned "accumulate slope" difference is correlated with the drift rate difference across subjects.
- Take the ERPs to source space and look for the source space signatures of the above. These could then be compared to the fMRI data.
- See if any of these power band signatures predict psychiatric dysfunction or other task-related questionnaire items (cognitive flexibility, impulsivity).

# Time-Frequency Power

In addition to comparing ERPs, we also look at differences in time-frequency power between the conditions. 

## Compute the TFR Power

### Compute the Raw Power

To compute the time-frequency power we use morlet wavelet convolution. Morlet wavelet convolution involves taking sinusoids of varying frequencies and windowing them with a gaussian window. These wavelets are then convolved with the data to produce time varying power estimates at the given frequency. Using a set of frequencies produces a time frequency power representation. The other important parameter is how many cycles of each frequency to include in the gaussian window. This parameter determines the temporal and frequency resolution tradeoff. More cycles give greater frequency resolution at the expense of temporal resolution and vice versa.

We followed the methods of <a href="https://www.ncbi.nlm.nih.gov/pubmed/24068756">Cohen and Donner, 2013</a>. Specifically, this included frequencies logarithimically spaced from 2-60 with accompanying number of cycles logarithmically spaced from 3-10. The log spacing gives equal weighting to the different frequency bands which are defined on a logarithmic scale. 

Additionally, following Cohen and Donner's finding that conflict-related theta is non-phase locked, we compute the non-phase locked power here. To do so, we remove the ERP for each condition from every trial separately for each condition. Thus, after the transformation we have a non-phase locked time-frequency representation for every channel, epoch pairing.

In [18]:
import sys
sys.path.append('../src')
from utils import select_subjects
from mne.time_frequency import tfr_morlet, write_tfrs
import numpy as np
from mne import read_epochs, set_log_level

set_log_level('critical')

subjects = select_subjects('both')

preproc_dir = '../data/derivatives/eeg_preprocessing'
deriv_dir = '../data/derivatives/eeg_analysis'

# select out the subjects
subjects = select_subjects('both')

# load config parameters
config = json.load(open('experiment_config.json', 'r'))
epochs_info = config['epochs_info']

# match Cohen, Donner 2013 
frequencies = np.logspace(np.log10(2), np.log10(60), num=30) 
n_cycles = np.logspace(np.log10(3), np.log10(10), num=30) 

for trans_type in ['base', 'laplacian']:
    print(trans_type)

    for epo_times, epo_type in zip(epochs_info['epo_boundaries'],
                                   epochs_info['epo_types']):
        print(epo_type)
        group_evo = {'incongruent': [], 'congruent': []}
        group_data = {'incongruent': [], 'congruent': []}

        for subject in subjects:
            print(subject)

            # load epochs
            if trans_type == 'laplacian':
                f = '%s/%s/%s_task-msit_%s_laplacian-epo.fif' 
            else:
                f = '%s/%s/%s_task-msit_%s-epo.fif'
            epochs = read_epochs(f % (preproc_dir, subject, subject,
                                      epo_type), verbose=False)

            # interpolate the bad channels
            epochs.interpolate_bads(reset_bads=True)

            # calculate power
            tfrs = []
            for event in config['event_types']:
                cond_epochs = epochs[event]

                # remove evoked response
                cond_epochs.subtract_evoked()

                # compute tfr
                power = tfr_morlet(cond_epochs, freqs=frequencies, 
                                   n_cycles=n_cycles, decim=5, 
                                   return_itc=False, average=False, n_jobs=2)

                # crop out filter buffer
                power.crop(epo_times[0], epo_times[1])
                tfrs.append(power)

            # save raw tfr
            if trans_type == 'laplacian':
                f = '%s/%s/%s_task-msit_%s_laplacian_raw-tfr.h5' 
            else:
                f = '%s/%s/%s_task-msit_%s_raw-tfr.h5'
                
            write_tfrs(f % (deriv_dir, subject, subject, epo_type), tfrs, 
                       overwrite=True)

print('Done!')

base
stimulus
sub-hc001
sub-hc002
sub-hc003
sub-hc004
sub-hc005
sub-hc006
sub-hc007
sub-hc008
sub-hc009
sub-hc010
sub-hc011
sub-hc012
sub-hc014
sub-hc015
sub-hc016
sub-hc017
sub-hc019
sub-hc021
sub-hc022
sub-hc023
sub-hc024
sub-hc025
sub-hc026
sub-hc028
sub-hc029
sub-hc030
sub-hc031
sub-hc032
sub-hc033
sub-hc034
sub-hc035
sub-hc036
sub-hc042
sub-hc044
sub-pp001
sub-pp002
sub-pp003
sub-pp004
sub-pp005
sub-pp006
sub-pp007
sub-pp008
sub-pp009
sub-pp010
sub-pp011
sub-pp012
sub-pp013
sub-pp014
sub-pp015
sub-pp016
response
sub-hc001
sub-hc002
sub-hc003
sub-hc004
sub-hc005
sub-hc006
sub-hc007
sub-hc008
sub-hc009
sub-hc010
sub-hc011
sub-hc012
sub-hc014
sub-hc015
sub-hc016
sub-hc017
sub-hc019
sub-hc021
sub-hc022
sub-hc023
sub-hc024
sub-hc025
sub-hc026
sub-hc028
sub-hc029
sub-hc030
sub-hc031
sub-hc032
sub-hc033
sub-hc034
sub-hc035
sub-hc036
sub-hc042
sub-hc044
sub-pp001
sub-pp002
sub-pp003
sub-pp004
sub-pp005
sub-pp006
sub-pp007
sub-pp008
sub-pp009
sub-pp010
sub-pp011
sub-pp012
sub-pp013
sub-pp0

### Baseline Normalize

Raw TFR power is difficult to work with since it is highly dependent on the magnitude of the signal. This poses two problems:
1. Lower frequencies are inherently higher magnitude than higher frequencies and thus higher frequencies will be masked by the lower frequencies.
2. Subject's data comes at varying magnitudes, but we want to compare task-related changes across subjects.

To alleviate these issues, we baseline normalize the data. We use a baseline period of -.5 to -.1 s prior to stimulus presentation. We use the <a href="https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3183439/"> Grandchamp and Delorme baseline normalization method </a> which involves:
- First each trial is normalized by the mean power across the entire trial.
- The data is averaged acrossed trials
- The baseline normalization value is calculated by taking the mean of the baseline period for each channel and frequency.
- The data for each channel and frequency is normalized by dividing by this value.
- The data are log transformed and multiplied by 10.

Thus, the power data now ranges from -inf to +inf where a value of 1 corresponds to a 100-fold increase in power relative to baseine.

In [19]:
import sys
sys.path.append('../src')
from utils import select_subjects
from eeg import baseline_normalize 
import numpy as np
from mne.time_frequency import tfr_morlet, read_tfrs, write_tfrs
from mne import grand_average

subjects = select_subjects('both')

preproc_dir = '../data/derivatives/eeg_preprocessing'
deriv_dir = '../data/derivatives/eeg_analysis'

# select out the subjects
subjects = select_subjects('both')

# load config parameters
config = json.load(open('experiment_config.json', 'r'))
epochs_info = config['epochs_info']
baseline_bounds = epochs_info['baseline_boundary']

# cache stimulus baseline periods 
baselines = {}

for trans_type in ['base', 'laplacian']:
    print(trans_type)
    
    for epo_time, epo_type in zip(epochs_info['epo_boundaries'],
                                  epochs_info['epo_types']):
        print(epo_type)

        group = {'incongruent': [], 'congruent': []}
        
        for subject in subjects:
            
            print(subject)

            if trans_type == 'laplacian':
                f = '%s/%s/%s_task-msit_%s_laplacian_raw-tfr.h5'
            else:
                f = '%s/%s/%s_task-msit_%s_raw-tfr.h5'
            tfrs = read_tfrs(f % (deriv_dir, subject, subject, epo_type))

            if epo_type == 'stimulus':
                baselines[subject] = {'incongruent': baseline_bounds, 
                                      'congruent': baseline_bounds}

            norm_tfrs = []
            for i, c in enumerate(config['event_types']):

                tfr, baseline = baseline_normalize(tfrs[i], 
                                                   baselines[subject][c],
                                                   func=np.mean, 
                                                   method='grandchamp')
                norm_tfrs.append(tfr)
                group[c].append(tfr)
                baselines[subject][c] = baseline


            if trans_type == 'laplacian':
                f = '%s/%s/%s_task-msit_%s_laplacian_norm-tfr.h5'
            else:
                f = '%s/%s/%s_task-msit_%s_norm-tfr.h5'
            write_tfrs(f % (deriv_dir, subject, subject, epo_type), 
                       norm_tfrs, overwrite=True)

        group_tfrs = [grand_average(group[c]) for c in config['event_types']]
        if trans_type == 'laplacian':
            f = '%s/group/group_task-msit_%s_laplacian_norm-tfr.h5'
        else:
            f = '%s/group/group_task-msit_%s_norm-tfr.h5'
        write_tfrs(f % (deriv_dir, epo_type), group_tfrs, overwrite=True)

print('Done!')

base
stimulus
sub-hc001
sub-hc002
sub-hc003
sub-hc004
sub-hc005
sub-hc006
sub-hc007
sub-hc008
sub-hc009
sub-hc010
sub-hc011
sub-hc012
sub-hc014
sub-hc015
sub-hc016
sub-hc017
sub-hc019
sub-hc021
sub-hc022
sub-hc023
sub-hc024
sub-hc025
sub-hc026
sub-hc028
sub-hc029
sub-hc030
sub-hc031
sub-hc032
sub-hc033
sub-hc034
sub-hc035
sub-hc036
sub-hc042
sub-hc044
sub-pp001
sub-pp002
sub-pp003
sub-pp004
sub-pp005
sub-pp006
sub-pp007
sub-pp008
sub-pp009
sub-pp010
sub-pp011
sub-pp012
sub-pp013
sub-pp014
sub-pp015
sub-pp016
response
sub-hc001
sub-hc002
sub-hc003
sub-hc004
sub-hc005
sub-hc006
sub-hc007
sub-hc008
sub-hc009
sub-hc010
sub-hc011
sub-hc012
sub-hc014
sub-hc015
sub-hc016
sub-hc017
sub-hc019
sub-hc021
sub-hc022
sub-hc023
sub-hc024
sub-hc025
sub-hc026
sub-hc028
sub-hc029
sub-hc030
sub-hc031
sub-hc032
sub-hc033
sub-hc034
sub-hc035
sub-hc036
sub-hc042
sub-hc044
sub-pp001
sub-pp002
sub-pp003
sub-pp004
sub-pp005
sub-pp006
sub-pp007
sub-pp008
sub-pp009
sub-pp010
sub-pp011
sub-pp012
sub-pp013
sub-pp0

### Compute Band Power

Finally, we compute power band data. Given the high dimensionality of tfr power data, it is often useful to reduce the full frequency range to frequency bands of interest. We define theta as 3-7 Hz, alpha as 7.5-12.5 Hz, and beta as 15-30 Hz. The band power is obtained by simpling averaging the normalized power across all frequencies that lie within a band's range.

In [1]:
import sys
sys.path.append('../src')
from utils import select_subjects
from eeg import baseline_normalize 
import numpy as np
from mne.time_frequency import tfr_morlet, read_tfrs, write_tfrs
from mne import grand_average

subjects = select_subjects('both')

preproc_dir = '../data/derivatives/eeg_preprocessing'
deriv_dir = '../data/derivatives/eeg_analysis'

# select out the subjects
subjects = select_subjects('both')

# load config parameters
config = json.load(open('experiment_config.json', 'r'))
epochs_info = config['epochs_info']
bands = config['power_bands']

# cache stimulus baseline periods 
baselines = {}

for trans_type in ['base', 'laplacian']:
    print(trans_type)
    
    for epo_time, epo_type in zip(epochs_info['epo_boundaries'],
                                  epochs_info['epo_types']):
        print(epo_type)

        group = {}
        for band in bands.keys():
            group[band] = {'incongruent': [], 'congruent': []}
        
        for subject in subjects:
            print(subject)
            if trans_type == 'laplacian':
                f = '%s/%s/%s_task-msit_%s_laplacian_norm-tfr.h5'
            else:
                f = '%s/%s/%s_task-msit_%s_norm-tfr.h5'
            tfrs = read_tfrs(f % (deriv_dir, subject, subject, epo_type))
            
            sub_data = {}
            for band in bands.keys():
                sub_data[band] = []

                times = tfrs[0].times
                chs = tfrs[0].ch_names
                freqs = tfrs[0].freqs
                band_ix = np.where(np.logical_and(freqs >= bands[band][0],
                                                  freqs <= bands[band][1]))[0]
                for i, c in enumerate(config['event_types']):
                    data = tfrs[i].data[:, band_ix, :].mean(axis=1)
                    group[band][c].append(data)
                    sub_data[band].append(data)

            if trans_type == 'laplacian':
                f = '%s/%s/%s_task-msit_%s_laplacian_band-tfr.npz'
            else:
                f = '%s/%s/%s_task-msit_%s_band-tfr.npz'
            np.savez_compressed(f % (deriv_dir, subject, subject, epo_type),
                                chs=chs, conditions=config['event_types'],
                                freqs=freqs, times=times, 
                                theta=np.array(sub_data['theta']),
                                beta=np.array(sub_data['beta']),
                                alpha=np.array(sub_data['alpha']))
            
        group_bands = {}
        for band in bands.keys():
            group_bands[band] = []
            for c in config['event_types']:
                group_bands[band].append(np.array(group[band][c]))
                
        if trans_type == 'laplacian':
            f = '%s/group/group_task-msit_%s_laplacian_band-tfr.npz'
        else:
            f = '%s/group/group_task-msit_%s_band-tfr.npz'
        np.savez_compressed(f % (deriv_dir, epo_type),
                            chs=chs, conditions=config['event_types'],
                            freqs=freqs, times=times, subjects=subjects,
                            theta=np.array(group_bands['theta']),
                            beta=np.array(group_bands['beta']),
                            alpha=np.array(group_bands['alpha']))
        del group
        del group_bands

print('Done!')

base
stimulus
sub-hc001
Reading ../data/derivatives/eeg_analysis/sub-hc001/sub-hc001_task-msit_stimulus_norm-tfr.h5 ...
sub-hc002
Reading ../data/derivatives/eeg_analysis/sub-hc002/sub-hc002_task-msit_stimulus_norm-tfr.h5 ...
sub-hc003
Reading ../data/derivatives/eeg_analysis/sub-hc003/sub-hc003_task-msit_stimulus_norm-tfr.h5 ...
sub-hc004
Reading ../data/derivatives/eeg_analysis/sub-hc004/sub-hc004_task-msit_stimulus_norm-tfr.h5 ...
sub-hc005
Reading ../data/derivatives/eeg_analysis/sub-hc005/sub-hc005_task-msit_stimulus_norm-tfr.h5 ...
sub-hc006
Reading ../data/derivatives/eeg_analysis/sub-hc006/sub-hc006_task-msit_stimulus_norm-tfr.h5 ...
sub-hc007
Reading ../data/derivatives/eeg_analysis/sub-hc007/sub-hc007_task-msit_stimulus_norm-tfr.h5 ...
sub-hc008
Reading ../data/derivatives/eeg_analysis/sub-hc008/sub-hc008_task-msit_stimulus_norm-tfr.h5 ...
sub-hc009
Reading ../data/derivatives/eeg_analysis/sub-hc009/sub-hc009_task-msit_stimulus_norm-tfr.h5 ...
sub-hc010
Reading ../data/deriva

sub-hc033
Reading ../data/derivatives/eeg_analysis/sub-hc033/sub-hc033_task-msit_response_norm-tfr.h5 ...
sub-hc034
Reading ../data/derivatives/eeg_analysis/sub-hc034/sub-hc034_task-msit_response_norm-tfr.h5 ...
sub-hc035
Reading ../data/derivatives/eeg_analysis/sub-hc035/sub-hc035_task-msit_response_norm-tfr.h5 ...
sub-hc036
Reading ../data/derivatives/eeg_analysis/sub-hc036/sub-hc036_task-msit_response_norm-tfr.h5 ...
sub-hc042
Reading ../data/derivatives/eeg_analysis/sub-hc042/sub-hc042_task-msit_response_norm-tfr.h5 ...
sub-hc044
Reading ../data/derivatives/eeg_analysis/sub-hc044/sub-hc044_task-msit_response_norm-tfr.h5 ...
sub-pp001
Reading ../data/derivatives/eeg_analysis/sub-pp001/sub-pp001_task-msit_response_norm-tfr.h5 ...
sub-pp002
Reading ../data/derivatives/eeg_analysis/sub-pp002/sub-pp002_task-msit_response_norm-tfr.h5 ...
sub-pp003
Reading ../data/derivatives/eeg_analysis/sub-pp003/sub-pp003_task-msit_response_norm-tfr.h5 ...
sub-pp004
Reading ../data/derivatives/eeg_anal

sub-hc002
Reading ../data/derivatives/eeg_analysis/sub-hc002/sub-hc002_task-msit_response_laplacian_norm-tfr.h5 ...
sub-hc003
Reading ../data/derivatives/eeg_analysis/sub-hc003/sub-hc003_task-msit_response_laplacian_norm-tfr.h5 ...
sub-hc004
Reading ../data/derivatives/eeg_analysis/sub-hc004/sub-hc004_task-msit_response_laplacian_norm-tfr.h5 ...
sub-hc005
Reading ../data/derivatives/eeg_analysis/sub-hc005/sub-hc005_task-msit_response_laplacian_norm-tfr.h5 ...
sub-hc006
Reading ../data/derivatives/eeg_analysis/sub-hc006/sub-hc006_task-msit_response_laplacian_norm-tfr.h5 ...
sub-hc007
Reading ../data/derivatives/eeg_analysis/sub-hc007/sub-hc007_task-msit_response_laplacian_norm-tfr.h5 ...
sub-hc008
Reading ../data/derivatives/eeg_analysis/sub-hc008/sub-hc008_task-msit_response_laplacian_norm-tfr.h5 ...
sub-hc009
Reading ../data/derivatives/eeg_analysis/sub-hc009/sub-hc009_task-msit_response_laplacian_norm-tfr.h5 ...
sub-hc010
Reading ../data/derivatives/eeg_analysis/sub-hc010/sub-hc010_t

## TFR Band Power TFCE Permutation Testing

The permutation process is exactly the same as the ERP permutation testing except here we additionally split by power band.

In [3]:
import os
import sys
import json
sys.path.append('../src')
from utils import select_subjects
import pandas as pd
import numpy as np
from mne import read_evokeds
from mne.channels import find_ch_connectivity
from mne.stats import spatio_temporal_cluster_1samp_test as stcluster_test

np.random.seed(10)

# input and output directories
deriv_dir = '../data/derivatives/eeg_analysis'

# select out the subjects
subjects = select_subjects('both')

# get the channel connectivity
f = '%s/group/group_task-msit_stimulus_laplacian-ave.fif'
evo = read_evokeds(f % deriv_dir, verbose=False)[0]
connectivity, ch_names = find_ch_connectivity(evo.info, 
                                              ch_type='eeg')

# load config parameters
config = json.load(open('experiment_config.json', 'r'))
epochs_info = config['epochs_info']
threshold = config['tfce_threshold'] 
num_perm = config['num_eeg_perm']

for trans_type in ['base', 'laplacian']:
    print(trans_type)
    
    for epo_type in epochs_info['epo_types']:
        print(epo_type)


        # extract the data (incongruent - congruent)
        if trans_type == 'laplacian':
            band_f = '%s/group/group_task-msit_%s_laplacian_band-tfr.npz'
        else:
            band_f = '%s/group/group_task-msit_%s_band-tfr.npz'
        
        for band in config['power_bands'].keys(): 
            print(band)
            
            data = np.load(band_f % (deriv_dir, epo_type))[band]
            data = data[0] - data[1]
            data = np.swapaxes(data, 1, 2)

            # run threshold free cluster enhancement permutation testing
            tfce, _, p_vals, perm_dist = stcluster_test(data, verbose=True,
                                                        threshold=threshold,
                                                        n_permutations=num_perm,
                                                        connectivity=connectivity,
                                                        seed=5, n_jobs=10) 
            p_vals = np.array(p_vals).reshape(tfce.shape)

            # save
            if trans_type == 'laplacian':
                f = '%s/group/group_task-msit_%s_%s_laplacian_tfce.npz'
            else:
                f = '%s/group/group_task-msit_%s_%s_tfce.npz'
            f = f % (deriv_dir, epo_type, band)
            np.savez_compressed(f, tfce=tfce.T, perm_dist=perm_dist, 
                                chs=evo.ch_names, times=evo.times, 
                                p_vals=p_vals.T)

print('Done!')

Could not find a connectivity matrix for the data. Computing connectivity based on Delaunay triangulations.
-- number of connected vertices : 70
base
stimulus
theta
stat_fun(H1): min=-6.109270 max=5.121132
Running initial clustering
Using 62 thresholds from 0.00 to 6.10 for TFCE computation (h_power=2.00, e_power=0.50)
Found 31570 clusters
Permuting 4999 times...
[......................................  ] 96.19238 \    

[Parallel(n_jobs=10)]: Done   3 out of  10 | elapsed: 62.1min remaining: 144.9min
[Parallel(n_jobs=10)]: Done   6 out of  10 | elapsed: 62.5min remaining: 41.7min


Computing cluster p-values


[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed: 63.7min finished


Done.
beta
stat_fun(H1): min=-11.414357 max=4.614588
Running initial clustering
Using 115 thresholds from 0.00 to 11.40 for TFCE computation (h_power=2.00, e_power=0.50)
Found 31570 clusters
Permuting 4999 times...
[......................................  ] 96.19238 \    

...

[......................................  ] 96.19238 \    

[Parallel(n_jobs=10)]: Done   6 out of  10 | elapsed: 59.1min remaining: 39.4min


Computing cluster p-values


[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed: 60.5min finished


Done.
Done!


## Visualize TFR Power

### TFR Heatmaps

The below cell plots tfr power heatmaps interactively. One can cycle through what subject to plot, the channel to plot, the color limits for the heat map, as well as whether to look at laplacian vs. non-laplacian transformed data.

In [1]:
import sys
sys.path.append('../src')
from utils import select_subjects
from eeg import visualize_tfr_heatmap, CH_NAMES
from ipywidgets import interact, fixed
import seaborn as sns
import pandas as pd
import json
sns.set(font_scale=2, style='whitegrid')

%matplotlib inline

subjects = ['group'] + select_subjects('both')

with open('experiment_config.json', 'r') as fid:
    config = json.load(fid)

deriv_dir = '../data/derivatives/eeg_analysis'

# load behavior
behavior = pd.read_csv('../data/derivatives/behavior/group_data.tsv',
                       sep='\t', na_values='n/a')

interact(visualize_tfr_heatmap, deriv_dir=fixed(deriv_dir), subject=subjects,
         trans_type=['laplacian', 'base'], ch=CH_NAMES, lim=(0, 3, .1),
         behavior=fixed(behavior), config=fixed(config));

aW50ZXJhY3RpdmUoY2hpbGRyZW49KERyb3Bkb3duKGRlc2NyaXB0aW9uPXUnc3ViamVjdCcsIG9wdGlvbnM9KCdncm91cCcsICdzdWItaGMwMDEnLCAnc3ViLWhjMDAyJywgJ3N1Yi1oYzAwMyfigKY=


### Band Power 

Here we visualize the group band power along with the statistical results. The cell below lets you interactively explore the data. Each parameter setting produces the following:
1. A band power waveform plot on the left:
    - The incongruent and congruent conditions are plotted separately with standard error of the mean shading. 
    - The RT distributions are plotted in the background for the stimulus-locked data. 
    - Signficant time points for that channel are denoted with a shaded grey background
    - A dotted line indicates the time point plotted for the topomap to the right
2. A band power topomap on the right:
    - This is the mean incongruent - congruent topomap signal plotted for just the chosen time point
    - Colorbar limits are hard to see. They vary from +- 1 dB 
    - Significant sensors for that timepoint have their names plotted in bold on the topomap

Use the following parameter controls to explore the data:
    - Use the ch parameter to change which channel's ERPs are plotted on the left
    - Use the trans_type and epo_type parameters to change what type of ERPs are plotted
    - Use time_ix parameter to change what time point is plotted for the topomap
    - Use the threshold parameter to change the significance highlighting
    - Use the band parameter to change the frequency band plotted

In [2]:
import sys
sys.path.append('../src')
from utils import select_subjects
from eeg import visualize_band_power, CH_NAMES
from ipywidgets import interact, fixed
import seaborn as sns
import json
import pandas as pd
sns.set(font_scale=2, style='whitegrid')

%matplotlib inline

with open('experiment_config.json', 'r') as fid:
    config = json.load(fid)
bands = config['power_bands']

# load behavior
behavior = pd.read_csv('../data/derivatives/behavior/group_data.tsv',
                       sep='\t', na_values='n/a')

deriv_dir = '../data/derivatives/eeg_analysis'
interact(visualize_band_power, deriv_dir=fixed(deriv_dir),
         trans_type=['laplacian', 'base'], epo_type=['stimulus', 'response'], 
         time_ix=(0, 450), ch=CH_NAMES, band=['theta', 'alpha', 'beta'],
         threshold=[.001, .01, .05], behavior=fixed(behavior));

aW50ZXJhY3RpdmUoY2hpbGRyZW49KERyb3Bkb3duKGRlc2NyaXB0aW9uPXUndHJhbnNfdHlwZScsIG9wdGlvbnM9KCdsYXBsYWNpYW4nLCAnYmFzZScpLCB2YWx1ZT0nbGFwbGFjaWFuJyksIETigKY=


## Summary of Power Results

Again, we see reasonable alignment with <a href="https://www.ncbi.nlm.nih.gov/pubmed/28220517">Gonzalez-Villar and Carillo-de-la-Pena, 2017</a>, one of the few published investigations of EEG in the MSIT task that we use as a reference:
- Greater midfrontal theta for incongruent compared to congruent trials. This seems to be concentrated above FCz for us as opposed to Cz for them. We found a similarly low theta (~2.5-7 Hz rather than a more common 4-8 Hz).
- A greater reduction in alpha for incongruent compared to congruent trials, especially concentrated over the parietal/occipital areas. This is clearly in line with incongruent trials requiring greater sustained attention as alpha is often regarded as a marker of attention/alertness.
- We also see reduced beta power in the incongruent vs. congruent trials over the central electrodes. As noted by Gonzalez-Villar, this could reflect greater sustained motor preparation for incongruent compared to congruent trials.

Alignment with RT Model Parameters:
- The FCz theta difference looks as if it could reflect a difference in decision boundary. One possibility is that this difference could be correlated with the decision boundary difference. This would make the case that the midfrontal theta accumulation represents some method of evidence accumulation or directed effort towards evidence accumulation.
- The occipital alpha difference could be related to the difference in drift rate. As alpha can be thought of as a proxy for attention, a greater reduction in alpha could be correlated with a slower drift rate in incongruent trials compared to congruent trials reflecting the added difficulty in accumulating information for incongruent trials. 

### Potential Concerns

- The same concerns with the TFCE statistical approach apply here though the issue appears to be less pronounced.
- Arbritrary drawing of power bands. We use the Gonzalez-Villar and Cohen papers to inform our power band limits, however as revealed with the heatmaps, the power differences are often fairly complex. One might look at attemping to uncover differences in the 2D power space rather than the 1D band space to get a more complete picture of the differences.

### Future Directions

- Fix the statistics.
- Look at behavior RT model correlations with power signatures.
- Source localize the band power and look for the source space signatures of these difference frequency band differences. It could also be interesting to compare spatially to the fMRI.
- Look for connectivity dynamics. Perhaps midfrontal theta serves as a control signal to parietal based accumulation and this could be reflected by theta coherence between the two.
- See if any of these power band signatures predict psychiatric dysfunction or other task-related questionnaire items (cognitive flexibility, impulsivity).
